In [2]:
import pandas as pd
import numpy as np 
import pymongo
from pymongo import MongoClient

In [3]:
conn = MongoClient(host='127.0.0.1', port=27017)  # 实例化MongoClient
db = conn.get_database('lianjia')  # 连接到Lianjia数据库
zufang = db.get_collection('zufang') # 连接到集合zufang
mon_data = zufang.find()  # 查询这个集合下的所有记录
lianjia_df=pd.DataFrame(list(mon_data))

In [4]:
import pyecharts
from pyecharts.charts import Bar
from pyecharts import options as opts

In [51]:
# 1.城市各地区房源数量  1.以区县分钟，统计数量

df = lianjia_df[lianjia_df['city'] == '广州']
apartment_data = df[df['dataDistributionType'] == 1].groupby('district')['price'].count()
entire_rent_data = df[df['title'].str.contains('整租')].groupby('district')['title'].count()
joint_rent_data = df[df['title'].str.contains('合租')].groupby('district')['area'].count()
df_new = pd.DataFrame([apartment_data, entire_rent_data, joint_rent_data])
df_new = df_new.T
df_new.rename(columns={'price': 'apartment', 'title': 'entire', 'area': 'joint'}, inplace=True) 
data_x = df_new.index
apartment_y_data = list(df_new.iloc[:, 0].astype("float"))
entire_rent_y_data = list(df_new.iloc[:, 1].astype("float"))
joint_rent_y_data = list(df_new.iloc[:, 2].astype("float"))
bar = (
        Bar(
            init_opts=opts.InitOpts(
                width="1000px"
                )
            )
            .add_xaxis(list(data_x))
            .add_yaxis('公寓', apartment_y_data)  # numpy.int32不能显示
            .add_yaxis('整租', entire_rent_y_data)
            .add_yaxis('合租', joint_rent_y_data)
            .set_global_opts(title_opts=opts.TitleOpts(title="{}各地区租房房源数量统计")
        )
)
bar.render('html/count_bar.html')

'F:\\作业\\毕业设计\\项目\\lianjiaWeb\\Mainapp\\data_analysis\\html\\count_bar.html'

In [6]:
# 各每平米租金
apartment_per_price = df_gz[df_gz['dataDistributionType'] == 1].groupby('district')['per_price'].mean().round(decimals=1)
geren_per_price = df_gz.loc[df_gz['dataDistributionType'] == 0].groupby('district')['per_price'].mean().round(decimals=1)
df = pd.DataFrame([geren_per_price, apartment_per_price])
df = df.T
y1 = df.iloc[:, 0].astype("float")
y2 = df.iloc[:, 1].astype("float")
bar2 = (
    Bar()
    .add_xaxis(list(df.index))
    .add_yaxis('个人房源',list(y1))  #numpy.int32不能显示
    .add_yaxis('公寓',list(y2))
    .set_global_opts(title_opts=opts.TitleOpts(title="广州各地区租房房源数量统计"))
)
bar2.render('./Mainapp/data_analysis/html/test3_pie.html')


In [30]:
# 户型分布
from pyecharts.charts import Pie
temp = df_gz.groupby('houseType')['price'].count().reset_index()
temp.rename(columns={'price': 'counter'}, inplace=True) 
temp = temp.nlargest(10, 'counter').reset_index()
x_data = list(temp['houseType'].values)
y_data = list(temp['counter'].values.astype("float"))
data_pair = [list(z) for z in zip(x_data, y_data)]
data_pair.sort(key=lambda x: x[1])
print(data_pair)

pie = (
    Pie()
    .add("", [list(z) for z in zip(x_data, y_data)])
    .set_global_opts(title_opts=opts.TitleOpts(title="广州户型分布"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
   
)
pie.render('./Mainapp/data_analysis/html/test3_pie.html')


[['4室1厅2卫', 712.0], ['3室1厅2卫', 713.0], ['4室2厅2卫', 1193.0], ['2室2厅1卫', 1961.0], ['3室1厅1卫', 2285.0], ['3室2厅1卫', 2573.0], ['1室0厅1卫', 3324.0], ['3室2厅2卫', 3424.0], ['1室1厅1卫', 4816.0], ['2室1厅1卫', 5448.0]]


'F:\\作业\\毕业设计\\项目\\lianjiaWeb\\Mainapp\\data_analysis\\html\\test3_pie.html'

In [48]:
# 朝向统计
temp = df_gz.dropna(subset=['orientation'])  # 清理数据
temp = temp.groupby('orientation')['price'].count().reset_index()
temp.rename(columns={'price': 'counter'}, inplace=True) 
temp = temp.nlargest(6, 'counter').reset_index()
x_data = list(temp['orientation'].values)
y_data = list(temp['counter'].values.astype("float"))
data_pair = [list(z) for z in zip(x_data, y_data)]
data_pair.sort(key=lambda x: x[1])
pie = (
    Pie()
    .add(
        series_name="",
        radius=["50%", "70%"],
        data_pair=data_pair,
        label_opts=opts.LabelOpts(is_show=False, position="center"),
    )
    .set_global_opts(legend_opts=opts.LegendOpts(pos_left="legft", orient="vertical"))
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
        label_opts=opts.LabelOpts(formatter="{b}:  ({d}%)")
    )
    
)
pie.render('./Mainapp/data_analysis/html/orientation_pie.html')


'F:\\作业\\毕业设计\\项目\\lianjiaWeb\\Mainapp\\data_analysis\\html\\orientation_pie.html'

In [104]:
# 租房面积分布
bins = [0, 20,  40,  60,  80,  100,  120,  140,  160,  180,  200, 350]
temp2 = df_gz.dropna(subset=['area'])  # 清理数据
temp2['area_group'] = pd.cut(temp2.area, bins, right=False)  # 面积分组
temp2 = temp2.groupby('area_group')['price'].count().reset_index()
temp2.rename(columns={'price': 'counter'}, inplace=True) 
temp2['area_group']
bar_area = (
    Bar()
    .add_xaxis(list(temp2['area_group'].values.astype('str')))
    .add_yaxis("面积统计", list(temp2['counter'].values.astype("float")))
    .set_global_opts(title_opts=opts.TitleOpts(title="Bar-基本示例", subtitle="我是副标题"))
    
)
bar_area.render('./Mainapp/data_analysis/html/area_group_bar.html')
    

'F:\\作业\\毕业设计\\项目\\lianjiaWeb\\Mainapp\\data_analysis\\html\\area_group_bar.html'

In [122]:
# 租金最高的十个商圈
temp3 = df_gz.groupby('microdistrict')['per_price'].mean().reset_index()
temp3 = temp3.round(1)
temp3 = temp3.nlargest(10, 'per_price').reset_index()
attr = list(temp3['microdistrict'])
value = list(temp3['per_price'])
bar_mic_top10 = (
    Bar(
        init_opts=opts.InitOpts(
            width="1000px", 
            animation_opts=opts.AnimationOpts(
                animation_delay=1000, animation_easing="elasticOut"
            )
        )
    )
    .add_xaxis(attr)
    .add_yaxis("", value)
    .set_global_opts(title_opts=opts.TitleOpts(title="Bar-动画配置基本示例", subtitle="我是副标题"))
)
bar_mic_top10.render('./Mainapp/data_analysis/html/bar_mic_top10.html')


'F:\\作业\\毕业设计\\项目\\lianjiaWeb\\Mainapp\\data_analysis\\html\\bar_mic_top10.html'

In [145]:
# 词云
from pyecharts.charts import WordCloud
from collections import Counter
tags = []
temp4 = df_gz.copy()
for tag in temp4.dropna(subset=['tags'])['tags']:
    tags.extend(tag)
for orientation, houseType in zip(temp4['orientation'], temp4['houseType']):
    tags.append(orientation)
    tags.append(houseType)

tag_name = Counter(tags).keys()
tag_count = Counter(tags).values()
wc = (
    WordCloud()
    .add(series_name="热点分析", data_pair=[list(z) for z in zip(tag_name, tag_count)], word_size_range=[6, 66])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="热点分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
)
wc.render('./Mainapp/data_analysis/html/worldCloud.html')


'F:\\作业\\毕业设计\\项目\\lianjiaWeb\\Mainapp\\data_analysis\\html\\worldCloud.html'

In [174]:
# 标签统计
from operator import itemgetter
tags2 = []
for tag in temp4.dropna(subset=['tags'])['tags']:
    tags2.extend(tag)
x_data = sorted(Counter(tags2).items(), key=itemgetter(1), reverse=True)
y_data = sorted(Counter(tags2).items(), key=itemgetter(1), reverse=True)
x_data = list(map(itemgetter(0), x_data))
y_data = list(map(itemgetter(1), y_data))
bar_wordCount = (
    Bar()
    .add_xaxis(list(x_data))
    .add_yaxis('', list(y_data))  
    .set_global_opts(
        title_opts=opts.TitleOpts(title="标签统计"),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-50)),
    )
)
bar_wordCount.render('./Mainapp/data_analysis/html/bar_wordCount.html')


'F:\\作业\\毕业设计\\项目\\lianjiaWeb\\Mainapp\\data_analysis\\html\\bar_wordCount.html'

In [71]:
# 租赁类型 饼状图
from pyecharts.charts import Pie
temp = lianjia_df[lianjia_df['city'] == '广州']
apartment_data = df[df['dataDistributionType'] == 1].count()
entire_rent_data = df[df['title'].str.contains('整租')].count()
joint_rent_data = df[df['title'].str.contains('合租')].count()
y_data = [float(apartment_data[0]), float(entire_rent_data[0]), float(joint_rent_data[0])]
x_data = ['公寓', '整租', '合租']
data_pair = [list(z) for z in zip(x_data, y_data)]
pie = (
    Pie()
    .add(
        series_name="",
        radius=["50%", "70%"],
        data_pair=data_pair,
        label_opts=opts.LabelOpts(is_show=False, position="center"),
    )
    .set_global_opts(legend_opts=opts.LegendOpts(pos_left="legft", orient="vertical"))
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
        label_opts=opts.LabelOpts(formatter="{b}:  ({d}%)")
    )
    
)
pie.render('html/rentType_pie.html')

'F:\\作业\\毕业设计\\项目\\lianjiaWeb\\Mainapp\\data_analysis\\html\\rentType_pie.html'